### Required Installations

In [1]:
# ! pip install python-dotenv
# !pip install --upgrade langchain
# !pip install langchain-community langchain-core
# !pip install -q -U google-generativeai
# !pip install -U langchain-openai
# !pip install -q -U google-generativeai
# !pip install -qU langchain-google-genai
# !pip install -U langchain-ollama
# !ollama pull llama3.1
# !pip install -qU langchain_mistralai


#### Imports

In [2]:


# from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate



from dotenv import load_dotenv, find_dotenv
import os
from  openai  import OpenAI
import pandas as pd
pd.set_option('display.max_colwidth', None)

#### Load Environment Variables

In [3]:
_ = load_dotenv(find_dotenv())
# print(os.environ["GOOGLE_API_KEY"])
# print(os.environ["LANGCHAIN_API_KEY"])
# print(os.environ["OPENAI_API_KEY"])


### Input csv file

In [4]:
file_name = "train_mini.csv"
# file_name = "train_full.csv"

df = pd.read_csv(file_name)
df.shape
conversation = df.iloc[1]['conversation']
print(conversation)

USER:Hi i need a help, i am very hungry, I am looking for a restaurant
SYSTEM:Sure, I will help you, What type of food are you looking for? Which city should i search in?
USER:Some Punjabi kind of foods in milpitas



#### Common Variable declarations and prompt template

In [5]:
prompt_template = """
Your role is to ask an agent to {intent}. 
Construct a one line user instruction based on this intention as first person from the conversation.

Focus on whether user is asking you to {intent} the name of the place,time and party size. 
Also identify required number of people, time , place or any other available entities and features to {intent} in the conversation.
Do not display these entities separately though.
Use this information in various order to construct the user instruction. Do not make up information if it is not availble.
Keep the same verb from conversation in the instruction that is used to {intent}.
Use diverse mood and personality in the instruction.
"""

intent_dict = {
    'ReserveRestaurant' : 'Reserve a Restaurant',
    'FindRestaurants':'Find a Restaurant',
    'SearchHotel': 'Search a Hotel',
    'ReserveHotel':'Reserve a Hotel'
}


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",f"{prompt_template}",
        ),
        ("human", "{conversation}"),
    ]
)

### Define LLM models


In [13]:
# Define LLM
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


from langchain_openai import ChatOpenAI
openai_llm = ChatOpenAI(
    model_name= "gpt-4o-mini", 
    temperature=0.5
    )


from langchain_ollama import ChatOllama

ollama_llm = ChatOllama(
    model = "llama3.1",
    temperature = 0.5,
    num_predict = 256,
    # other params ...
)


from langchain_mistralai import ChatMistralAI

mistral_llm = ChatMistralAI(
    model="open-mistral-7b",
    temperature=0.5,
    # max_retries=2,
    # other params...
)


In [12]:
import time

openai_chain = prompt | openai_llm
gemini_chain = prompt | gemini_llm
ollama_chain = prompt | ollama_llm
mistral_chain = prompt | mistral_llm
model_chains = {
    'gpt': openai_chain,
    'gemini':gemini_chain,
    # 'mistral':mistral_chain,
    # 'ollama':ollama_chain
    }


experiment = True

for model in model_chains.keys():
    chain = model_chains[model]
    results = []
    # # Experimentation : Iterate over each row
    if experiment:
        for i in  range(1,205,10):
            dialogue_id = df.iloc[i]['dialogue_id']
            conversation = df.iloc[i]['conversation']
            intent = intent_dict[df.iloc[i]['service_call_method']]
            if  intent != 'Reserve a Restaurant':continue

            # Apply LLM chain on the desired column in each row
            result = chain.invoke(
                {
                "conversation":dialogue_id +': '+ conversation,
                "intent":intent
                }
            )
            results.append(result.content)
            time.sleep(5)

    else:   

        # # Iterate over each row
        for index, row in df.iterrows():
            # Apply LLM chain on the desired column in each row
            result = chain.invoke({
                'conversation' : row['dialogue_id']+': '+row['conversation'],
                'intent' :intent_dict[row['service_call_method']]
                })

            results.append(result.content)
            break
            
    print(f"{model}: {results}")

gpt: ["I'd like to reserve a table for 2 at Bird Dog in Palo Alto for 11:30 am today, please!", 'Please reserve a table for 2 at the Grand Harbor in Burlingame today at 12:45 pm.', 'Can you reserve a table for 4 at Hunan Chili in Mountain View for 6 in the evening?', "I'd like to make a reservation for two people at Lake Chalet in Oakland for 10:30 this morning.", 'Can you please reserve a table for 4 at Soo Yuan Restaurant in Calistoga at 1:30 this afternoon?', "I'd like to reserve a table for three at Chaat Corner in San Francisco on March 14th at 7:30 pm, please."]


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


gemini: ['Can you please make a booking at this restaurant for me at 11:30 am? \n', 'Please make a reservation at the Grand Harbor in Burlingame for quarter to 1 in the afternoon today. \n', 'I want to reserve a table for 4 at 6 in the evening at Hunan Chili in Mountain View.  \n', 'Can you please make a reservation for two people at Lake Chalet in Oakland at 10:30 in the morning? \n', 'Please make reservation for 4 including me at Soo Yuan Restaurant in Calistoga at 1:30 pm today. \n', 'Can you make a reservation for me at Chaat Corner in San Francisco for three people on March 14th at 7:30 pm? \n']


In [ ]:
# df['instruction_openai']=results
openai_chain.


SyntaxError: invalid syntax (121049500.py, line 2)

In [ ]:
df_out=df[['dialogue_id','service_call_method','conversation','instruction','service_call_parameters']]

# Using Gemini AI

In [ ]:


gemini_chain = prompt | gemini_llm




In [ ]:
result.content

In [ ]:
import time
# Experimentation : Iterate over each row
for i in  range(1,205,20):
    dialogue_id = df.iloc[i]['dialogue_id']
    conversation = df.iloc[i]['conversation']
    intent = intent_dict[df.iloc[i]['service_call_method']]
    if  intent != 'Reserve a Restaurant':continue

    # Apply LLM chain on the desired column in each row
    result = gemini_chain.invoke(  
        {
            "intent": intent,
            "conversation":dialogue_id +': '+ conversation,
        }
    )
    print(result.content)
    time.sleep(5)


